# Analysis for Build Components, CogSci 2024

In [6]:
import os
import sys
import urllib, io
import pickle

import random
import numpy as np
import scipy.stats as stats
import pandas as pd
from sklearn.metrics import euclidean_distances, jaccard_score, pairwise_distances

import pymongo as pm
from collections import Counter
import json
import re
import ast

from PIL import Image, ImageOps, ImageDraw, ImageFont 
from IPython.core.display import HTML 

from io import BytesIO
import base64
import requests

import  matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
from matplotlib.colors import ListedColormap

import seaborn as sns
sns.set_context('talk')
sns.set_style('darkgrid')

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

sys.path.append("../../stimuli/block_utils/")
import blockworld_utils as utils
import domino_settings as dominoes
import domino_settings_monochrome as mono_dominoes

import importlib


In [17]:
experiment_name = 'build_components/cogsci24'

## directory & file hierarchy
proj_dir = os.path.abspath('../..')
datavol_dir = os.path.join(proj_dir,'data')
analysis_dir = os.path.abspath(os.path.join(os.getcwd(),'..'))
results_dir = os.path.join(proj_dir,'results')

# paths specific to this experiment
experiment_results_dir = os.path.join(results_dir, experiment_name)
plot_dir = os.path.join(experiment_results_dir,'plots')
csv_dir = os.path.join(experiment_results_dir,'csv')

## add helpers to python path
if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       

In [13]:
# plot settings

monochrome = True
block_settings = mono_dominoes if monochrome else dominoes
importlib.reload(block_settings)

block_dims = block_settings.block_dims
block_colors = block_settings.block_colors
world_width = block_settings.world_width
world_height = block_settings.world_height
world_center = block_settings.world_center
black = ['#000000','#000000'] # used to display silhouettes
grey = ['#333333','#333333']

In [14]:
# sns.set_context('poster')
# sns.set(font_scale=1.2)
sns.set_style('whitegrid', {'font.family':'sans-serif', 'font.sans-serif':'Avenir'})
import poster_plot_settings
from poster_plot_settings import palette

## read in data

We read in data from 4 sets of csvs, consisting of:  
- 2 encoding conditions: **Visual Exposure (ve)** and **Working Memory (wm)**.  
- 2 decoding conditions: **Recognition/ old-new (on)** and **Building Recall (re)**.  

"Experiments" reported in our paper contain both decoding conditions for each encoding condition i.e. Experiment 1 is VE and Experiment 2 is WM.

In [33]:
# final iteration names are suffixes of csvs

csv_ids = {
    "ve_on" : "build_components_cogsci_ve_old_new_prolific_pilot_0",
    "ve_re": "build_components_cogsci_ve_recall_prolific_pilot_0",
    "wm_on" : "build_components_cogsci_wm_old_new_prolific_pilot_0",
    "wm_re": "build_components_cogsci_wm_recall_prolific_pilot_0"
}

In [34]:
# read in all dataframes

dfs = {}

for i, (k, v) in enumerate(csv_ids.items()):
    dfs[k] = {}
    dfs[k]['encode'] = pd.read_csv(experiment_results_dir + '/df_encode_{}.csv'.format(v))
    dfs[k]['decode'] = pd.read_csv(experiment_results_dir + '/df_decode_{}.csv'.format(v))
    dfs[k]['block'] = pd.read_csv(experiment_results_dir + '/df_block_{}.csv'.format(v))
    dfs[k]['construct'] = pd.read_csv(experiment_results_dir + '/df_construction_procedure_{}.csv'.format(v))
    
dfs['ve_re']['recalls'] = pd.read_csv(experiment_results_dir + 
                                '/df_recalled_towers_{}.csv'.format(csv_ids['ve_re']))
dfs['wm_re']['recalls'] = pd.read_csv(experiment_results_dir + 
                                '/df_recalled_towers_{}.csv'.format(csv_ids['wm_re']))

# Visual Exposure Experiment

# Working Memory Experiment